# Multifidelity Modelling

### [Neil D. Lawrence](http://inverseprobability.com), University of

Cambridge

### 2021-11-09

**Abstract**: This week we introduce multifidelity modelling. We use
surrogate models to capture different qualities of information from
different simulations.

$$
$$

<!-- Do not edit this file locally. -->
<!-- Do not edit this file locally. -->

p

<!---->
<!-- Do not edit this file locally. -->
<!-- Do not edit this file locally. -->
<!-- The last names to be defined. Should be defined entirely in terms of macros from above-->
<!--

-->

## Setup

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_notebooks/includes/notebook-setup.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_notebooks/includes/notebook-setup.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})

<!--setupplotcode{import seaborn as sns
sns.set_style('darkgrid')
sns.set_context('paper')
sns.set_palette('colorblind')}-->

## notutils

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_software/includes/notutils-software.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_software/includes/notutils-software.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

This small package is a helper package for various notebook utilities
used

The software can be installed using

In [ ]:
%pip install notutils

from the command prompt where you can access your python installation.

The code is also available on GitHub:
<https://github.com/lawrennd/notutils>

Once `notutils` is installed, it can be imported in the usual manner.

In [ ]:
import notutils

## mlai

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_software/includes/mlai-software.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_software/includes/mlai-software.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

The `mlai` software is a suite of helper functions for teaching and
demonstrating machine learning algorithms. It was first used in the
Machine Learning and Adaptive Intelligence course in Sheffield in 2013.

The software can be installed using

In [ ]:
%pip install mlai

from the command prompt where you can access your python installation.

The code is also available on GitHub: <https://github.com/lawrennd/mlai>

Once `mlai` is installed, it can be imported in the usual manner.

In [ ]:
import mlai

In [ ]:
%pip install gpy

In [ ]:
%pip install pyDOE

In [ ]:
%pip install emukit

# An Introduction to Multi-fidelity Modeling in Emukit

A reminder from our lecture on Emulation. This diagram implies that we
might expect our statistical emulator to be able to ‘adjudicate’ between
simulations with different fidelity.

<img src="https://mlatcl.github.io/mlphysical/./slides/diagrams//uq/statistical-emulation004.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>A statistical emulator is a system that reconstructs the
simulation with a statistical model. As well as reconstructing the
simulation, a statistical emulator can be used to correlate with the
real world.</i>

## Overview

This section is based on the [Emukit multifidelity tutorial found
here](https://github.com/EmuKit/emukit/blob/master/notebooks/Emukit-tutorial-multi-fidelity-bayesian-optimization.ipynb)
and written by Javier Gonzalez, Mark Pullin, Oleg Ponomarev and
David-Elias Künstle.

A common issue encountered when applying machine learning to
environmental sciences and engineering problems is the difficulty or
cost required to obtain sufficient data for building robust models.
Possible examples include aerospace and nautical engineering, where it
is both infeasible and prohibitively expensive to run a vast number of
experiments using the actual vehicle. Even when there is no physical
artifact involved, such as in climate modeling, data may still be hard
to obtain when these can only be collected by running an expensive
computer experiment, where the time required to acquire an individual
data sample restricts the volume of data that can later be used for
modeling.

Constructing a reliable model when only few observations are available
is challenging, which is why it is common practice to develop
*simulators* of the actual system, from which data points can be more
easily obtained. In engineering applications, such simulators often take
the form of Computational Fluid Dynamics (CFD) tools which approximate
the behaviour of the true artifact for a given design or configuration.
However, although it is now possible to obtain more data samples, it is
highly unlikely that these simulators model the true system exactly;
instead, these are expected to contain some degree of bias and/or noise.

From the above, one can deduce that naively combining observations from
multiple information sources could result in the model giving biased
predictions which do not accurately reflect the true problem. To this
end, *multi-fidelity models* are designed to augment the limited true
observations available with cheaply-obtained approximations in a
principled manner. In such models, observations obtained from the true
source are referred to as *high-fidelity* observations, whereas
approximations are denoted as being *low-fidelity*. These low-fidelity
observations are then systemically combined with the more accurate (but
limited) observations in order to predict the high-fidelity output more
effectively. Note than we can generally combine information from
multiple lower fidelity sources, which can all be seen as auxiliary
tasks in support of a single primary task.

In this notebook, we shall investigate a selection of multi-fidelity
models based on Gaussian processes which are readily available in
`EmuKit`. We start by investigating the traditional linear
multi-fidelity model as proposed in (Kennedy and O’Hagan, 2000).
Subsequently, we shall illustrate why this model can be unsuitable when
the mapping from low to high-fidelity observations is nonlinear, and
demonstrate how an alternate model proposed in Paris Perdikaris et al.
(2017) can alleviate this issue. The examples presented in this notebook
can then be easily adapted to a variety of problem settings.

## Linear multi-fidelity model

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_uq/includes/emukit-linear-multifidelity.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_uq/includes/emukit-linear-multifidelity.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

The linear multi-fidelity model proposed in Kennedy and O’Hagan (2000)
is widely viewed as a reference point for all such models. In this
model, the high-fidelity (true) function is modeled as a scaled sum of
the low-fidelity function plus an error term: $$
f_{\text{high}}(x) = f_{\text{err}}(x) + \rho \,f_{\text{low}}(x)
$$ In this equation, $f_{\text{low}}(x)$ is taken to be a Gaussian
process modeling the outputs of the lower fidelity function, while
$\rho$ is a scaling factor indicating the magnitude of the correlation
to the high-fidelity data. Setting this to 0 implies that there is no
correlation between observations at different fidelities. Meanwhile,
$f_{\text{err}}(x)$ denotes yet another Gaussian process which models
the bias term for the high-fidelity data. Note that $f_{\text{err}}(x)$
and $f_{\text{low}}(x)$ are assumed to be independent processes which
are only related by the equation given above.

**Note**: While we shall limit our explanation to the case of two
fidelities, this set-up can easily be generalized to cater for $T$
fidelities as follows: $$
f_{t}(x) = f_{t}(x) + \rho_{t-1} \,f_{t-1}(x), \quad t=1,\dots, T
$$

If the training points are sorted such that the low and high-fidelity
points are grouped together: $$
\begin{pmatrix}
\mathbf{X}_{\text{low}} \\
\mathbf{X}_{\text{high}}
\end{pmatrix}
$$

we can express the model as a single Gaussian process having the
following prior. $$
\begin{bmatrix}
f_{\text{low}}\left(h\right)\\
f_{\text{high}}\left(h\right)
\end{bmatrix}
\sim
GP
\begin{pmatrix}
\begin{bmatrix}
0 \\ 0
\end{bmatrix},
\begin{bmatrix}
k_{\text{low}} & \rho k_{\text{low}} \\
\rho k_{\text{low}} & \rho^2 k_{\text{low}} + k_{\text{err}}
\end{bmatrix}
\end{pmatrix}
$$

### Linear multi-fidelity modeling in Emukit

As a first example of how the linear multi-fidelity model implemented in
`Emukit` `emukit.multi_fidelity.models.GPyLinearMultiFidelityModel` can
be used, we shall consider the two-fidelity Forrester function. This
benchmark is frequently used to illustrate the capabilities of
multi-fidelity models.

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)

In [ ]:
np.random.seed(20)

In [ ]:
import GPy
import emukit.multi_fidelity
import emukit.test_functions
from emukit.model_wrappers.gpy_model_wrappers import GPyMultiOutputWrapper
from emukit.multi_fidelity.models import GPyLinearMultiFidelityModel

Generate samples from the Forrester function

In [ ]:
high_fidelity = emukit.test_functions.forrester.forrester
low_fidelity = emukit.test_functions.forrester.forrester_low

x_plot = np.linspace(0, 1, 200)[:, np.newaxis]
y_plot_l = low_fidelity(x_plot)
y_plot_h = high_fidelity(x_plot)

x_train_l = np.atleast_2d(np.random.rand(12)).T
x_train_h = np.atleast_2d(np.random.permutation(x_train_l)[:6])
y_train_l = low_fidelity(x_train_l)
y_train_h = high_fidelity(x_train_h)

The inputs to the models are expected to take the form of ndarrays where
the last column indicates the fidelity of the observed points.

Although only the input points, $X$, are augmented with the fidelity
level, the observed outputs $Y$ must also be converted to array form.

For example, a dataset consisting of 3 low-fidelity points and 2
high-fidelity points would be represented as follows, where the input is
three-dimensional while the output is one-dimensional: $$
\mathbf{X}= \begin{pmatrix}
x_{\text{low};0}^0 & x_{\text{low};0}^1 & x_{\text{low};0}^2 & 0\\
x_{\text{low};1}^0 & x_{\text{low};1}^1 & x_{\text{low};1}^2 & 0\\
x_{\text{low};2}^0 & x_{\text{low};2}^1 & x_{\text{low};2}^2 & 0\\
x_{\text{high};0}^0 & x_{\text{high};0}^1 & x_{\text{high};0}^2 & 1\\
x_{\text{high};1}^0 & x_{\text{high};1}^1 & x_{\text{high};1}^2 & 1
\end{pmatrix}\quad
\mathbf{Y}= \begin{pmatrix}
y_{\text{low};0}\\
y_{\text{low};1}\\
y_{\text{low};2}\\
y_{\text{high};0}\\
y_{\text{high};1}
\end{pmatrix}
$$ This is a representation we first developed for the `GPy` software.
It allows for a lot of flexibility for Gaussian processes that describe
multiple correlated functions, like the ‘multi-fidelity’ model of
Kennedy and O’Hagan (2000).

As an aside there is quite a lot of history to modelling Gaussian
processes which represent multiple output functions. Back in 2009, with
Mauricio Alvarez, we organized a series of workshops where we worked
across the geostatistics, the emulation and the machine learning
communities to build understanding. You can see [details of the first of
these workshops (held in Manchester)
here](http://gpss.cc/slim09/schedule.html). The second of these
workshops also integrated ideas from the classical kernel community, and
was held at NeurIPS in 2009 in collaboration with [Lorenzo
Rosasco](https://www.dibris.unige.it/en/rosasco-lorenzo), you can find
the [workshop page here](http://gpss.cc/mock09/). We summarized the
conclusions from those meetings in a [review paper led by Mauricio, that
you can find here](https://arxiv.org/abs/1106.6251) (Álvarez et al.,
2012).

In that terminology the multifidelity model we’re using here is known as
a “intrinsic coregionalisation model” and it is one of the simplest
types of multi-output Gaussian processes you can build.

Mauricio’s thesis (Álvarez, 2011) focused on particular multiple output
covariances derived from physical information embedded in the system,
such as differential equations. See e.g., Álvarez et al. (2013) or
Lawrence et al. (n.d.) for an application.

A similar procedure must be carried out for obtaining predictions at new
test points, whereby the fidelity indicated in the column then indicates
the fidelity at which the function must be predicted for a designated
point.

For convenience of use, we provide helper methods for easily converting
between a list of arrays (ordered from the lowest to the highest
fidelity) and the required ndarray representation. This is found in
`emukit.multi_fidelity.convert_lists_to_array`.

Convert lists of arrays to ndarrays augmented with fidelity indicators.

In [ ]:
from emukit.multi_fidelity.convert_lists_to_array import convert_x_list_to_array, convert_xy_lists_to_arrays

In [ ]:
X_train, Y_train = convert_xy_lists_to_arrays([x_train_l, x_train_h], 
                                                    [y_train_l, y_train_h])

Plot the original functions.

In [ ]:
import matplotlib.pyplot as plt
import mlai.plot as plot
import mlai

In [ ]:
fig, ax = plt.subplots(figsize=plot.big_wide_figsize)

ax.plot(x_plot, y_plot_l, 'b', linewidth=3)
ax.plot(x_plot, y_plot_h, 'r', linewidth=3)
ax.scatter(x_train_l, y_train_l, color='b', s=40)
ax.scatter(x_train_h, y_train_h, color='r', s=40)
ax.set_ylabel('$f(x)$')
ax.set_xlabel('$x$')
ax.legend(['Low fidelity', 'High fidelity'])

mlai.write_figure('high-and-low-fidelity-forrester.svg', directory='./uq')

<img src="https://mlatcl.github.io/mlphysical/./slides/diagrams//uq/high-and-low-fidelity-forrester.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>High and low fidelity Forrester functions.</i>

Observe that in the example above we restrict our observations to 12
from the lower-fidelity function and only 6 from the high-fidelity
function. As we shall demonstrate further below, fitting a standard GP
model to the few high-fidelity observations is unlikely to result in an
acceptable fit, which is why we shall instead consider the linear
multi-fidelity model presented in this section.

Below we fit a linear multi-fidelity model to the available low and
high-fidelity observations. Given the smoothness of the functions, we
opt to use an exponentiated quadratic kernel for both the bias and
correlation components of the model.

**Note**: The model implementation defaults to a `MixedNoise` noise
likelihood whereby there is independent Gaussian noise for each
fidelity.

This can be modified upfront using the ‘likelihood’ parameter in the
model constructor, or by updating them directly after the model has been
created. In the example below, we choose to fix the noise to ‘0’ for
both fidelities to reflect that the observations are exact.

Construct a linear multi-fidelity model.

In [ ]:
kernels = [GPy.kern.RBF(1), GPy.kern.RBF(1)]
lin_mf_kernel = emukit.multi_fidelity.kernels.LinearMultiFidelityKernel(kernels)
gpy_lin_mf_model = GPyLinearMultiFidelityModel(X_train, Y_train, lin_mf_kernel, n_fidelities=2)
gpy_lin_mf_model.mixed_noise.Gaussian_noise.fix(0)
gpy_lin_mf_model.mixed_noise.Gaussian_noise_1.fix(0)

Wrap the model using the given `GPyMultiOutputWrapper`

In [ ]:
lin_mf_model = GPyMultiOutputWrapper(gpy_lin_mf_model, 2, n_optimization_restarts=5)

Fit the model

In [ ]:
lin_mf_model.optimize()

Convert x_plot to its ndarray representation.

In [ ]:
X_plot = convert_x_list_to_array([x_plot, x_plot])
X_plot_l = X_plot[:len(x_plot)]
X_plot_h = X_plot[len(x_plot):]

Compute mean predictions and associated variance.

In [ ]:
lf_mean_lin_mf_model, lf_var_lin_mf_model = lin_mf_model.predict(X_plot_l)
lf_std_lin_mf_model = np.sqrt(lf_var_lin_mf_model)
hf_mean_lin_mf_model, hf_var_lin_mf_model = lin_mf_model.predict(X_plot_h)
hf_std_lin_mf_model = np.sqrt(hf_var_lin_mf_model)

Plot the posterior mean and variance.

In [ ]:
import matplotlib.pyplot as plt
import mlai.plot as plot
import mlai

In [ ]:
fig, ax = plt.subplots(figsize=plot.big_wide_figsize)
ax.fill_between(x_plot.flatten(), (lf_mean_lin_mf_model - 1.96*lf_std_lin_mf_model).flatten(), 
                 (lf_mean_lin_mf_model + 1.96*lf_std_lin_mf_model).flatten(), facecolor='g', alpha=0.3)
ax.fill_between(x_plot.flatten(), (hf_mean_lin_mf_model - 1.96*hf_std_lin_mf_model).flatten(), 
                 (hf_mean_lin_mf_model + 1.96*hf_std_lin_mf_model).flatten(), facecolor='y', alpha=0.3)

ax.plot(x_plot, y_plot_l, 'b', linewidth=3)
ax.plot(x_plot, y_plot_h, 'r', linewidth=3)
ax.plot(x_plot, lf_mean_lin_mf_model, '--', color='g', linewidth=3)
ax.plot(x_plot, hf_mean_lin_mf_model, '--', color='y', linewidth=3)
ax.scatter(x_train_l, y_train_l, color='b', s=40)
ax.scatter(x_train_h, y_train_h, color='r', s=40)
ax.set_ylabel('$f(x)$')
ax.set_xlabel('$x$')
ax.legend(['Low Fidelity', 'High Fidelity', 'Predicted Low Fidelity', 'Predicted High Fidelity'])

mlai.write_figure('linear-multi-fidelity-model.svg', directory='./uq')

<img src="https://mlatcl.github.io/mlphysical/./slides/diagrams//uq/linear-multi-fidelity-model.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>Linear multi-fidelity model fit to low and high-fidelity
Forrester function</i>

The above plot demonstrates how the multi-fidelity model learns the
relationship between the low and high-fidelity observations to model
both of the corresponding functions.

In this example, the posterior mean almost fits the true function
exactly, while the associated uncertainty returned by the model is also
appropriately small given the good fit.

### Comparison to standard GP

In the absence of such a multi-fidelity model, a regular Gaussian
process would have been fit exclusively to the high fidelity data.

As illustrated in the figure below, the resulting Gaussian process
posterior yields a much worse fit to the data than that obtained by the
multi-fidelity model. The uncertainty estimates are also poorly
calibrated.

Create standard GP model using only high-fidelity data.

In [ ]:
kernel = GPy.kern.RBF(1)
high_gp_model = GPy.models.GPRegression(x_train_h, y_train_h, kernel)
high_gp_model.Gaussian_noise.fix(0)

Fit the GP model.

In [ ]:
high_gp_model.optimize_restarts(5)

Compute mean predictions and associated variance.

In [ ]:
hf_mean_high_gp_model, hf_var_high_gp_model  = high_gp_model.predict(x_plot)
hf_std_hf_gp_model = np.sqrt(hf_var_high_gp_model)

Plot the posterior mean and variance for the high-fidelity GP model.

In [ ]:
import matplotlib.pyplot as plt
import mlai.plot as plot
import mlai

In [ ]:
fig, ax = plt.subplots(figsize=plot.big_wide_figsize)

ax.fill_between(x_plot.flatten(), (hf_mean_lin_mf_model - 1.96*hf_std_lin_mf_model).flatten(), 
                 (hf_mean_lin_mf_model + 1.96*hf_std_lin_mf_model).flatten(), facecolor='y', alpha=0.3)
ax.fill_between(x_plot.flatten(), (hf_mean_high_gp_model - 1.96*hf_std_hf_gp_model).flatten(), 
                 (hf_mean_high_gp_model + 1.96*hf_std_hf_gp_model).flatten(), facecolor='k', alpha=0.1)

ax.plot(x_plot, y_plot_h, color='r', linewidth=3)
ax.plot(x_plot, hf_mean_lin_mf_model, '--', color='y', linewidth=3)
ax.plot(x_plot, hf_mean_high_gp_model, 'k--', linewidth=3)
ax.scatter(x_train_h, y_train_h, color='r')
ax.set_xlabel('$x$')
ax.set_ylabel('$f(x)$')
ax.legend(['True Function', 'Linear Multi-fidelity GP', 'High fidelity GP'])

mlai.write_figure('linear-multi-fidelity-high-fidelity-gp.svg', directory='./uq')

<img src="https://mlatcl.github.io/mlphysical/./slides/diagrams//uq/linear-multi-fidelity-high-fidelity-gp.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>Comparison of linear multi-fidelity model and high fidelity
GP</i>

## Nonlinear multi-fidelity model

Although the model described above works well when the mapping between
the low and high-fidelity functions is linear, several issues may be
encountered when this is not the case.

Consider the following example, where the low and high-fidelity
functions are defined as follows: $$
f_{\text{low}}(x) = \sin(8\pi x)
$$

$$
f_{\text{high}}(x) = \left(x- \sqrt{2}\right) \, f_{\text{low}}^2
$$

Generate data for nonlinear example.

In [ ]:
high_fidelity = emukit.test_functions.non_linear_sin.nonlinear_sin_high
low_fidelity = emukit.test_functions.non_linear_sin.nonlinear_sin_low

In [ ]:
x_plot = np.linspace(0, 1, 200)[:, np.newaxis]
y_plot_l = low_fidelity(x_plot)
y_plot_h = high_fidelity(x_plot)

n_low_fidelity_points = 50
n_high_fidelity_points = 14

x_train_l = np.linspace(0, 1, n_low_fidelity_points)[:, np.newaxis]
y_train_l = low_fidelity(x_train_l)

x_train_h = x_train_l[::4, :]
y_train_h = high_fidelity(x_train_h)

Convert lists of arrays to `ND-array`s augmented with fidelity
indicators

In [ ]:
X_train, Y_train = convert_xy_lists_to_arrays([x_train_l, x_train_h], [y_train_l, y_train_h])

In [ ]:
import matplotlib.pyplot as plt
import mlai.plot as plot
import mlai

In [ ]:
fig, ax = plt.subplots(figsize=plot.big_wide_figsize)

ax.plot(x_plot, y_plot_l, 'b', linewidth=3)
ax.plot(x_plot, y_plot_h, 'r', linewidth=3)
ax.scatter(x_train_l, y_train_l, color='b', s=40)
ax.scatter(x_train_h, y_train_h, color='r', s=40)
ax.set_xlabel('$x$')
ax.set_ylabel('$f(x)$')
ax.set_xlim([0, 1])
ax.legend(['Low fidelity', 'High fidelity'])

mlai.write_figure('high-and-low-fidelity-functions.svg', directory='./uq')

<img src="https://mlatcl.github.io/mlphysical/./slides/diagrams//uq/high-and-low-fidelity-functions.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>High and low-fidelity functions</i>

In this case, the mapping between the two functions is nonlinear, as can
be observed by plotting the high-fidelity observations as a function of
the lower fidelity observations.

In [ ]:
import matplotlib.pyplot as plt
import mlai.plot as plot
import mlai

In [ ]:
fig, ax = plt.subplots(figsize=plot.big_wide_figsize)
ax.set_ylabel('HF(x)')
ax.set_xlabel('LF(x)')
ax.plot(y_plot_l, y_plot_h, color=colors['purple'], linewidth=3)
ax.legend(['HF-LF Correlation'], loc='lower center')

mlai.write_figure('mapping-low-to-high-fidelity.svg', directory='./uq')

<img src="https://mlatcl.github.io/mlphysical/./slides/diagrams//uq/mapping-low-to-high-fidelity.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>Mapping from low-fidelity to high-fidelity.</i>

### Failure of linear multi-fidelity model

Below we fit the linear multi-fidelity model to this new problem and
plot the results.

Construct a linear multi-fidelity model.

In [ ]:
kernels = [GPy.kern.RBF(1), GPy.kern.RBF(1)]
lin_mf_kernel = emukit.multi_fidelity.kernels.LinearMultiFidelityKernel(kernels)
gpy_lin_mf_model = GPyLinearMultiFidelityModel(X_train, Y_train, lin_mf_kernel, n_fidelities=2)
gpy_lin_mf_model.mixed_noise.Gaussian_noise.fix(0)
gpy_lin_mf_model.mixed_noise.Gaussian_noise_1.fix(0)

lin_mf_model = model = GPyMultiOutputWrapper(gpy_lin_mf_model, 2, n_optimization_restarts=5)

Fit the model

In [ ]:
lin_mf_model.optimize()

Convert test points to appropriate representation

In [ ]:
X_plot = convert_x_list_to_array([x_plot, x_plot])
X_plot_low = X_plot[:200]
X_plot_high = X_plot[200:]

Compute mean and variance predictions

In [ ]:
hf_mean_lin_mf_model, hf_var_lin_mf_model = lin_mf_model.predict(X_plot_high)
hf_std_lin_mf_model = np.sqrt(hf_var_lin_mf_model)

Compare linear and nonlinear model fits

In [ ]:
import matplotlib.pyplot as plt
import mlai.plot as plot
import mlai

In [ ]:
fig, ax = plt.subplots(figsize=plot.big_wide_figsize)
ax.plot(x_plot, y_plot_h, 'r', linewidth=3)
ax.plot(x_plot, hf_mean_lin_mf_model, '--', color='y', linewidth=3)
ax.scatter(x_train_h, y_train_h, color='r')
ax.fill_between(x_plot.flatten(), (hf_mean_lin_mf_model - 1.96*hf_std_lin_mf_model).flatten(), 
                 (hf_mean_lin_mf_model + 1.96*hf_std_lin_mf_model).flatten(), color='y', alpha=0.3)
ax.set_xlim(0, 1)
ax.set_xlabel('$x$')
ax.set_ylabel('$f(x)$')
ax.legend(['True Function', 'Linear multi-fidelity GP'], loc='lower right')
mlai.write_figure('linear-multi-fidelity-model-fit.svg', directory='./uq/')

<img src="https://mlatcl.github.io/mlphysical/./slides/diagrams//uq/linear-multi-fidelity-model-fit.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>Linear multi-fidelity model fit to high-fidelity function</i>

As expected, the linear multi-fidelity model was unable to capture the
nonlinear relationship between the low and high-fidelity data.
Consequently, the resulting fit of the true function is also poor.

### Nonlinear Multi-fidelity model

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_uq/includes/emukit-nonlinear-multifidelity.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_uq/includes/emukit-nonlinear-multifidelity.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

In view of the deficiencies of the linear multi-fidelity model, a
nonlinear multi-fidelity model is proposed in Paris Perdikaris et al.
(2017) to better capture these correlations. This nonlinear model is
constructed as follows: $$ 
f_{\text{high}}(x) = \rho( \, f_{\text{low}}(x)) + \delta(x) 
$$ Replacing the linear scaling factor with a non-deterministic function
results in a model which can thus capture the nonlinear relationship
between the fidelities.

This model is implemented in Emukit as
`emukit.multi_fidelity.models.NonLinearModel`.

It is defined in a sequential manner where a Gaussian process model is
trained for every set of fidelity data available. Once again, we
manually fix the noise parameter for each model to 0. The parameters of
the two Gaussian processes are then optimized sequentially, starting
from the low-fidelity.

Create nonlinear model.

In [ ]:
from emukit.multi_fidelity.models.non_linear_multi_fidelity_model import make_non_linear_kernels, NonLinearMultiFidelityModel

In [ ]:
base_kernel = GPy.kern.RBF
kernels = make_non_linear_kernels(base_kernel, 2, X_train.shape[1] - 1)
nonlin_mf_model = NonLinearMultiFidelityModel(X_train, Y_train, n_fidelities=2, kernels=kernels, 
                                              verbose=True, optimization_restarts=5)
for m in nonlin_mf_model.models:
    m.Gaussian_noise.variance.fix(0)

In [ ]:
nonlin_mf_model.optimize()

Now we compute the mean and variance predictions

In [ ]:
hf_mean_nonlin_mf_model, hf_var_nonlin_mf_model = nonlin_mf_model.predict(X_plot_high)
hf_std_nonlin_mf_model = np.sqrt(hf_var_nonlin_mf_model)

lf_mean_nonlin_mf_model, lf_var_nonlin_mf_model = nonlin_mf_model.predict(X_plot_low)
lf_std_nonlin_mf_model = np.sqrt(lf_var_nonlin_mf_model)

In [ ]:
import matplotlib.pyplot as plt
import mlai.plot as plot
import mlai

In [ ]:
fig, ax = plt.subplots(figsize=plot.big_wide_figsize)
ax.fill_between(x_plot.flatten(), (lf_mean_nonlin_mf_model - 1.96*lf_std_nonlin_mf_model).flatten(), 
                 (lf_mean_nonlin_mf_model + 1.96*lf_std_nonlin_mf_model).flatten(), color='g', alpha=0.3)
ax.fill_between(x_plot.flatten(), (hf_mean_nonlin_mf_model - 1.96*hf_std_nonlin_mf_model).flatten(), 
                 (hf_mean_nonlin_mf_model + 1.96*hf_std_nonlin_mf_model).flatten(), color='y', alpha=0.3)
ax.plot(x_plot, y_plot_l, 'b', linewidth=3)
ax.plot(x_plot, y_plot_h, 'r', linewidth=3)
ax.plot(x_plot, lf_mean_nonlin_mf_model, '--', color='g', linewidth=3)
ax.plot(x_plot, hf_mean_nonlin_mf_model, '--', color='y', linewidth=3)
ax.scatter(x_train_h, y_train_h, color='r')
ax.set_xlabel('$x$')
ax.set_ylabel('$f(x)$')
ax.set_xlim(0, 1)
ax.legend(['Low Fidelity', 'High Fidelity', 'Predicted Low Fidelity', 'Predicted High Fidelity'])

mlai.write_figure('nonlinear-multi-fidelity-model-fit.svg', directory='./uq')

<img src="https://mlatcl.github.io/mlphysical/./slides/diagrams//uq/nonlinear-multi-fidelity-model-fit.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>Nonlinear multi-fidelity model fit to low and high-fidelity
functions.</i>

Fitting the nonlinear fidelity model to the available data very closely
fits the high-fidelity function while also fitting the low-fidelity
function exactly. This is a vast improvement over the results obtained
using the linear model. We can also confirm that the model is properly
capturing the correlation between the low and high-fidelity observations
by plotting the mapping learned by the model to the true mapping shown
earlier.

In [ ]:
import matplotlib.pyplot as plt
import mlai.plot as plot
import mlai

In [ ]:
fig, ax = plt.subplots(figsize=plot.big_wide_figsize)
ax.plot(y_plot_l, y_plot_h, '-', color=colors['purple'], linewidth=3)
ax.plot(lf_mean_nonlin_mf_model, hf_mean_nonlin_mf_model, 'k--', linewidth=3)
ax.set_ylabel('$HF(x)$')
ax.set_xlabel('$LF(x)$')
ax.legend(['True HF-LF Correlation', 'Learned HF-LF Correlation'], loc='lower center')

mlai.write_figure('mapping-low-fidelity-to-high-fidelity.svg', directory='./uq')

<img src="https://mlatcl.github.io/mlphysical/./slides/diagrams//uq/mapping-low-fidelity-to-high-fidelity.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>Mapping from low fidelity to high fidelity</i>

## Deep Gaussian Processes

<svg viewBox="0 0 200 200" style="width:15%">

<defs> <clipPath id="clip0">

<style>
circle {
  fill: black;
}
</style>

<circle cx="100" cy="100" r="100"/> </clipPath> </defs>

<title>

Andreas Damianou

</title>

<image preserveAspectRatio="xMinYMin slice" width="100%" xlink:href="https://mlatcl.github.io/mlphysical/./slides/diagrams//people/andreas-damianou.png" clip-path="url(#clip0)"/>

</svg>

These non-linear multi-fidelity models are an example of composing
Gaussian processes together. This type of non-linear relationship leads
to what we refer to as a Deep Gaussian process (Damianou and Lawrence,
2013; Lawrence and Moore, 2007) which Andreas Damianou worked on for his
PhD thesis (Damianou, 2015).

These ideas lead to the notion of ‘deep emulation,’ where a number of
emulators are chained together to represent a system.

## Deep Emulation

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_uq/includes/deep-emulation.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_uq/includes/deep-emulation.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

<img src="https://mlatcl.github.io/mlphysical/./slides/diagrams//simulation/ml-system-downstream-simulation000.svg" class="" width="75%" style="vertical-align:middle;">

Figure: <i>A potential path of models in the emulation of a simulation
system.</i>

As a solution we can use of *emulators*. When constructing an ML system,
software engineers, ML engineers, economists and operations researchers
are explicitly defining relationships between variables of interest in
the system. That implicitly defines a joint distribution,
$p(\mathbf{ y}^*, \mathbf{ y})$. In a decomposable system any
sub-component may be defined as
$p(\mathbf{ y}_\mathbf{i}|\mathbf{ y}_\mathbf{j})$ where
$\mathbf{ y}_\mathbf{i}$ and $\mathbf{ y}_\mathbf{j}$ represent sub-sets
of the full set of variables
$\left\{\mathbf{ y}^*, \mathbf{ y}\right\}$. In those cases where the
relationship is deterministic, the probability density would collapse to
a vector-valued deterministic function,
$\mathbf{ f}_\mathbf{i}\left(\mathbf{ y}_\mathbf{j}\right)$.

Inter-variable relationships could be defined by, for example a neural
network (machine learning), an integer program (operational research),
or a simulation (supply chain). This makes probabilistic inference in
this joint density for real world systems is either very hard or
impossible.

Emulation is a form of meta-modelling: we construct a model of the
model. We can define the joint density of an emulator as
$s(\mathbf{ y}*, \mathbf{ y})$, but if this probability density is to be
an accurate representation of our system, it is likely to be
prohibitively complex. Current practice is to design an emulator to deal
with a specific question. This is done by fitting an ML model to a
simulation from the the appropriate conditional distribution,
$p(\mathbf{ y}_\mathbf{i}|\mathbf{ y}_\mathbf{j})$, which is
intractable. The emulator provides an approximated answer of the form
$s(\mathbf{ y}_\mathbf{i}|\mathbf{ y}_\mathbf{j})$. Critically, an
emulator should incorporate its uncertainty about its approximation. So
the emulator answer will be less certain than direct access to the
conditional $p(\mathbf{ y}_i|\mathbf{ y}_j)$, but it may be sufficiently
confident to act upon. Careful design of emulators to answer a given
question leads to efficient diagnostics and understanding of the system.
But in a complex interacting system an exponentially increasing number
of questions can be asked. This calls for a system of automated
construction of emulators which selects the right structure and
redeploys the emulator as necessary. Rapid redeployment of emulators
could exploit pre-existing emulators through *transfer learning*.

Automatically deploying these families of emulators for full system
understanding is highly ambitious. It requires advances in engineering
infrastructure, emulation, and Bayesian optimization. However, the
intermediate steps of developing this architecture also allow for
automated monitoring of system accuracy and fairness. This facilitates
AutoML on a component-wise basis which we can see as a simple
implementation of AutoAI. The proposal is structured so that despite its
technical ambition there is a smooth ramp of benefits to be derived
across the program of work.

In Applied Mathematics, the field studying these techniques is known as
*uncertainty quantification*. The new challenge is the automation of
emulator creation on demand to answer questions of interest and
facilitate the system design, i.e. AutoAI through BSO.

At design stage, any AI task could be decomposed in multiple ways.
Bayesian system optimization will assist both in determining the
large-scale system design through exploring different decompositions and
in refinement of the deployed system.

So far, most work on emulators has focused on emulating a single
component. Automated deployment and maintenance of ML systems requires
networks of emulators that can be deployed and redeployed on demand
depending on the question of interest. Therefore, the technical
innovations we require are in the mathematical composition of emulator
models (Damianou and Lawrence, 2013; Paris Perdikaris et al., 2017).
Different chains of emulators will need to be rapidly composed to make
predictions of downstream performance. This requires rapid retraining of
emulators and *propagation of uncertainty* through the emulation
pipeline a process we call *deep emulation*.

This structural learning allows us to associate data with the relevant
layer of the model, rather than merely on the leaf nodes of the output
model. When deploying the deep Gaussian process as an emulator, this
allows for the possibility of learning the structure of the different
component parts of the underlying system. This should aid the user in
determining the ideal system decomposition.

<img src="https://mlatcl.github.io/mlphysical/./slides/diagrams//simulation/ml-system-downstream-simulation001.svg" class="" width="75%" style="vertical-align:middle;">

Figure: <i>A potential path of models in a machine learning system.</i>

<img src="https://mlatcl.github.io/mlphysical/./slides/diagrams//simulation/ml-system-downstream-simulation002.svg" class="" width="75%" style="vertical-align:middle;">

Figure: <i>A potential path of models in the emulation of a simulation
system.</i>

<img src="https://mlatcl.github.io/mlphysical/./slides/diagrams//simulation/ml-system-downstream-simulation003.svg" class="" width="75%" style="vertical-align:middle;">

Figure: <i>A potential path of models in the emulation of a simulation
system.</i>

## Brief Reflection

In this module, we have been introducing various aspects of surrogate
modelling. We’ve already seen in the sensitivity analysis section, how
we used experimental design to make our acquisition of data for the
Catapult simulator more efficient. To round of the taught session of the
course, we’ll also combine ideas from Bayesian optimization, with an
emulator built through experimental design.

The task is a classic example from reinforcement learning, known as the
‘Mountain Car.’ The idea is to drive an underpowered car up a hill. The
car doesn’t have the ability to accelerate hard enough, but it can build
momentum by oscillating up and down a hill to get to the target.

We provide some wrappers of the OpenAI Gym version of the mountain car
simulation in a python file. We will use this example to combine various
ideas from surrogate modelling to solve the problem.

<img src="https://mlatcl.github.io/mlphysical/./slides/diagrams//uq/mountain-car-emulation.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>The mountain car example contains a simulation of a car’s
dynamics within the wider simulation of the mountain. The simulation of
the car is called as a subroutine many times by the wider simulation of
the mountain. We can choose to build a surrogate model of the car, and
work with a modified mountain simulation where the emulator is called
instead of the car’s simulation directly.</i>

## Mountain Car Simulator

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_uq/includes/mountain-car-simulation.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_uq/includes/mountain-car-simulation.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

To illustrate the above mentioned concepts we use the [mountain car
simulator](https://github.com/openai/gym/wiki/MountainCarContinuous-v0).
This simulator is widely used in machine learning to test reinforcement
learning algorithms. The goal is to define a control policy on a car
whose objective is to climb a mountain. Graphically, the problem looks
as follows:

<img class="negate" src="https://mlatcl.github.io/mlphysical/./slides/diagrams//uq/mountaincar.png" style="width:60%">

Figure: <i>The mountain car simulation from the Open AI gym.</i>

The goal is to define a sequence of actions (push the car right or left
with certain intensity) to make the car reach the flag after a number
$T$ of time steps.

At each time step $t$, the car is characterized by a vector
$\mathbf{ x}_{t} = (p_t,v_t)$ of states which are respectively the the
position and velocity of the car at time $t$. For a sequence of states
(an episode), the dynamics of the car is given by

$$
\mathbf{ x}_{t+1} = f(\mathbf{ x}_{t},\textbf{u}_{t})
$$

where $\textbf{u}_{t}$ is the value of an action force, which in this
example corresponds to push car to the left (negative value) or to the
right (positive value). The actions across a full episode are
represented in a policy $\textbf{u}_{t} = \pi(\mathbf{ x}_{t},\theta)$
that acts according to the current state of the car and some parameters
$\theta$. In the following examples we will assume that the policy is
linear which allows us to write $\pi(\mathbf{ x}_{t},\theta)$ as

## Mountain Car Set Up

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_uq/includes/mountain-car-setup.py" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_uq/includes/mountain-car-setup.py', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

To run the mountain car example we need to install a python file that
we’ll download.

In [ ]:
import urllib.request

In [ ]:
urllib.request.urlretrieve('https://raw.githubusercontent.com/lawrennd/talks/gh-pages/mountain_car.py','mountain_car.py')

And to render the environment, the `pyglet` library.

In [ ]:
%pip install pyglet

$$
\pi(\mathbf{ x},\theta)= \theta_0 + \theta_p p + \theta_vv.
$$ For $t=1,\dots,T$ now given some initial state $\mathbf{ x}_{0}$ and
some some values of each $\textbf{u}_{t}$, we can **simulate** the full
dynamics of the car for a full episode using
[Gym](https://gym.openai.com/envs/). The values of $\textbf{u}_{t}$ are
fully determined by the parameters of the linear controller.

After each episode of length $T$ is complete, a reward function
$R_{T}(\theta)$ is computed. In the mountain car example, the reward is
computed as 100 for reaching the target of the hill on the right hand
side, minus the squared sum of actions (a real negative to push to the
left and a real positive to push to the right) from start to goal. Note
that our reward depends on $\theta$ as we make it dependent on the
parameters of the linear controller.

## Emulate the Mountain Car

In [ ]:
%pip install gym

In [ ]:
import gym

In [ ]:
env = gym.make('MountainCarContinuous-v0')

Our goal in this section is to find the parameters $\theta$ of the
linear controller such that

$$
\theta^* = arg \max_{\theta} R_T(\theta).
$$

In this section, we directly use Bayesian optimization to solve this
problem. We will use [EmuKit](https://emukit.github.io) so we first
define the objective function.

In [ ]:
import mountain_car as mc
import numpy as np

In [ ]:
def target_function(X):
    """Run the Mountain Car simulaton for each set of controller parameters in the matrix."""
    simulation_function = lambda x: mc.run_simulation(env, x)[0]
    return np.asarray([simulation_function(np.atleast_2d(x)) for x in X])[:, np.newaxis]

For each set of parameter values of the linear controller we can run an
episode of the simulator (that we fix to have a horizon of $T=500$) to
generate the reward. Using as input the parameters of the controller and
as outputs the rewards we can build a Gaussian process emulator of the
reward.

We start defining the input space, which is three-dimensional:

In [ ]:
from emukit.core import ContinuousParameter, ParameterSpace

In [ ]:
position_domain = [-1.2, +1]
velocity_domain = [-1/0.07, +1/0.07]
constant_domain = [-1, +1]

space = ParameterSpace(
          [ContinuousParameter('position_parameter', *position_domain), 
           ContinuousParameter('velocity_parameter', *velocity_domain),
           ContinuousParameter('constant', *constant_domain)])

To initalize the model we start sampling some initial points for the
linear controller randomly.

In [ ]:
from emukit.core.initial_designs import RandomDesign

In [ ]:
design = RandomDesign(space)
n_initial_points = 25
initial_design = design.get_samples(n_initial_points)

Now run the simulation 25 times across our initial design.

In [ ]:
y = target_function(initial_design)

Before we start any optimization, lets have a look to the behaviour of
the car with the first of these initial points that we have selected
randomly.

In [ ]:
import numpy as np

This won’t render in Google `colab`, but should work in a regular
Jupyter notebook if `pyglet` is installed. Details on rendering in
`colab` are given in answer to this stackoverflow question
<https://stackoverflow.com/questions/50107530/how-to-render-openai-gym-in-google-colab>.

In [ ]:
random_controller = initial_design[0,:]
_, _, _, frames = mc.run_simulation(env, np.atleast_2d(random_controller), render=True)
anim=mc.animate_frames(frames, 'Random linear controller')

In [ ]:
from IPython.core.display import HTML

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
mc.save_frames(frames, 
               diagrams='./uq', 
               filename='mountain-car-random.html')

<iframe src="https://mlatcl.github.io/mlphysical/./slides/diagrams//uq/mountain-car-random.html" width="600" height="450" allowtransparency="true" frameborder="0">
</iframe>

Figure: <i>Random linear controller for the Mountain car. It fails to
move the car to the top of the mountain.</i>

As we can see the random linear controller does not manage to push the
car to the top of the mountain. Now, let’s optimize the regret using
Bayesian optimization and the emulator for the reward. We try 50 new
parameters chosen by the expected improvement acquisition function.

First, we initizialize a Gaussian process emulator.

In [ ]:
import GPy

In [ ]:
kern = GPy.kern.RBF(3)
model_gpy = GPy.models.GPRegression(initial_design, y, kern, noise_var=1e-10)

In [ ]:
from emukit.model_wrappers.gpy_model_wrappers import GPyModelWrapper

In [ ]:
model_emukit = GPyModelWrapper(model_gpy, n_restarts=5)
model_emukit.optimize()

In Bayesian optimization an acquisition function is used to balance
exploration and exploitation to evaluate new locations close to the
optimum of the objective. In this notebook we select the expected
improvement (EI). For further details have a look at the review paper of
Shahriari et al. (2016).

In [ ]:
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement

In [ ]:
acquisition = ExpectedImprovement(model_emukit)

In [ ]:
from emukit.bayesian_optimization.loops.bayesian_optimization_loop import BayesianOptimizationLoop

In [ ]:
bo = BayesianOptimizationLoop(space, model_emukit, acquisition=acquisition)
bo.run_loop(target_function, 50)
results= bo.get_results()

Now we visualize the result for the best controller that we have found
with Bayesian optimization.

In [ ]:
_, _, _, frames = mc.run_simulation(env, np.atleast_2d(results.minimum_location), render=True)
anim=mc.animate_frames(frames, 'Best controller after 50 iterations of Bayesian optimization')

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
mc.save_frames(frames, 
               diagrams='./uq', 
               filename='mountain-car-simulated.html')

<iframe src="https://mlatcl.github.io/mlphysical/./slides/diagrams//uq/mountain-car-simulated.html" width="600" height="450" allowtransparency="true" frameborder="0">
</iframe>

Figure: <i>Mountain car simulator trained using Bayesian optimization
and the simulator of the dynamics. Fifty iterations of Bayesian
optimization are used to optimize the controler.</i>

The car can now make it to the top of the mountain! Emulating the reward
function and using expected improvement acquisition helped us to find a
linear controller that solves the problem.

## Data Efficient Emulation

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_uq/includes/mountain-car-data-efficient.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_uq/includes/mountain-car-data-efficient.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

In the previous section we solved the mountain car problem by directly
emulating the reward but no considerations about the dynamics $$
\mathbf{ x}_{t+1} =g(\mathbf{ x}_{t},\textbf{u}_{t})
$$ of the system were made.

We ran the simulator 25 times in the initial design, and 50 times in our
Bayesian optimization loop. That required us to call the dynamics
simulation $500\times 75 =37,500$ times, because each simulation of the
car used 500 steps. In this section we will show how it is possible to
reduce this number by building an emulator for $g(\cdot)$ that can later
be used to directly optimize the control.

The inputs of the model for the dynamics are the velocity, the position
and the value of the control so create this space accordingly.

In [ ]:
import gym

In [ ]:
env = gym.make('MountainCarContinuous-v0')

In [ ]:
from emukit.core import ContinuousParameter, ParameterSpace

In [ ]:
position_dynamics_domain = [-1.2, +0.6]
velocity_dynamics_domain = [-0.07, +0.07]
action_dynamics_domain = [-1, +1]

space_dynamics = ParameterSpace(
          [ContinuousParameter('position_dynamics_parameter', *position_dynamics_domain), 
           ContinuousParameter('velocity_dynamics_parameter', *velocity_dynamics_domain),
           ContinuousParameter('action_dynamics_parameter', *action_dynamics_domain)])

Next, we sample some input parameters and use the simulator to compute
the outputs. Note that in this case we are not running the full
episodes, we are just using the simulator to compute $\mathbf{ x}_{t+1}$
given $\mathbf{ x}_{t}$ and $\textbf{u}_{t}$.

In [ ]:
from emukit.core.initial_designs import RandomDesign

In [ ]:
design_dynamics = RandomDesign(space_dynamics)
n_initial_points = 500
initial_design_dynamics = design_dynamics.get_samples(n_initial_points)

In [ ]:
import numpy as np
import mountain_car as mc

In [ ]:
### --- Simulation of the (normalized) outputs
y_dynamics = np.zeros((initial_design_dynamics.shape[0], 2))
for i in range(initial_design_dynamics.shape[0]):
    y_dynamics[i, :] = mc.simulation(initial_design_dynamics[i, :])

In [ ]:
# Normalize the data from the simulation
y_dynamics_normalisation = np.std(y_dynamics, axis=0)
y_dynamics_normalised = y_dynamics/y_dynamics_normalisation

The outputs are the velocity and the position. Our model will capture
the change in position and velocity on time. That is, we will model

$$
\Delta v_{t+1} = v_{t+1} - v_{t}
$$

$$
\Delta x_{t+1} = p_{t+1} - p_{t}
$$

with Gaussian processes with prior mean $v_{t}$ and $p_{t}$
respectively. As a covariance function, we use `Matern52`. We need
therefore two models to capture the full dynamics of the system.

In [ ]:
import GPy

In [ ]:
kern_position = GPy.kern.Matern52(3)
position_model_gpy = GPy.models.GPRegression(initial_design_dynamics, y_dynamics[:, 0:1], kern_position, noise_var=1e-10)

In [ ]:
kern_velocity = GPy.kern.Matern52(3)
velocity_model_gpy = GPy.models.GPRegression(initial_design_dynamics, y_dynamics[:, 1:2], kern_velocity, noise_var=1e-10)

In [ ]:
from emukit.model_wrappers.gpy_model_wrappers import GPyModelWrapper

In [ ]:
position_model_emukit = GPyModelWrapper(position_model_gpy, n_restarts=5)
velocity_model_emukit = GPyModelWrapper(velocity_model_gpy, n_restarts=5)

In general, we might use much smarter strategies to design our emulation
of the simulator. For example, we could use the variance of the
predictive distributions of the models to collect points using
uncertainty sampling, which will give us a better coverage of the space.
For simplicity, we move ahead with the 500 randomly selected points.

Now that we have a data set, we can update the emulators for the
location and the velocity.

In [ ]:
position_model_emukit.optimize()
velocity_model_emukit.optimize()

We can now have a look to how the emulator and the simulator match.
First, we show a contour plot of the car acceleration for each pair of
can position and velocity. You can use the bar bellow to play with the
values of the controller to compare the emulator and the simulator.

In [ ]:
from IPython.html.widgets import interact

In [ ]:
control = mc.plot_control(velocity_model_emukit)
interact(control.plot_slices, control=(-1, 1, 0.05))

We can see how the emulator is doing a fairly good job approximating the
simulator. On the edges, however, it struggles to captures the dynamics
of the simulator.

Given some input parameters of the linear controlling, how do the
dynamics of the emulator and simulator match? In the following figure we
show the position and velocity of the car for the 500 time-steps of an
episode in which the parameters of the linear controller have been fixed
beforehand. The value of the input control is also shown.

In [ ]:
# change the values of the linear controller to observe the trajectories.
controller_gains = np.atleast_2d([0, .6, 1])  

In [ ]:
mc.emu_sim_comparison(env, controller_gains, 
                      [position_model_emukit, velocity_model_emukit], 
                      max_steps=500, diagrams='./uq')

<img src="https://mlatcl.github.io/mlphysical/./slides/diagrams//uq/emu-sim-comparison.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>Comparison between the mountain car simulator and the
emulator.</i>

We now make explicit use of the emulator, using it to replace the
simulator and optimize the linear controller. Note that in this
optimization, we don’t need to query the simulator anymore as we can
reproduce the full dynamics of an episode using the emulator. For
illustrative purposes, in this example we fix the initial location of
the car.

We define the objective reward function in terms of the simulator.

In [ ]:
### --- Optimize control parameters with emulator
car_initial_location = np.asarray([-0.58912799, 0])

In [ ]:
def target_function_emulator(X):
    """Run the Mountain Car simulation for each set of controller parameters in the matrix using the emulation."""
    emulation_function = lambda x: mc.run_emulation([position_model_emukit, velocity_model_emukit], x, car_initial_location)[0]
    return np.asarray([emulation_function(np.atleast_2d(x)) for x in X])[:, np.newaxis]

And as before, we use Bayesian optimization to find the best possible
linear controller.

The design space is the three continuous variables that make up the
linear controller.

In [ ]:
position_domain = [-1.2, +1]
velocity_domain = [-1/0.07, +1/0.07]
constant_domain = [-1, +1]

space = ParameterSpace(
          [ContinuousParameter('position_parameter', *position_domain), 
           ContinuousParameter('velocity_parameter', *velocity_domain),
           ContinuousParameter('constant', *constant_domain)])

<!--

::: {.cell .code}

```{.python}

:::{space= \[{‘name’:‘linear_1,’ ‘type’:‘continuous,’ ‘domain’:(-1/1.2,
+1)}, {‘name’:‘linear_2,’ ‘type’:‘continuous,’ ‘domain’:(-1/0.07,
+1/0.07)}, {‘name’:‘constant,’ ‘type’:‘continuous,’ ‘domain’:(-1,
+1)}\]–\>

    ::: {.cell .code}
    ``` {.python}
    from emukit.core.initial_designs import RandomDesign

:::

In [ ]:
design = RandomDesign(space)
n_initial_points = 25
initial_design = design.get_samples(n_initial_points)

Now run the simulation 25 times across our initial design.

In [ ]:
y = target_function_emulator(initial_design)

Now we set up the surrogate model for the Bayesian optimization loop.

In [ ]:
import GPy

In [ ]:
kern = GPy.kern.RBF(3)
model_dynamics_emulated_gpy = GPy.models.GPRegression(initial_design, y, kern, noise_var=1e-10)

In [ ]:
from emukit.model_wrappers.gpy_model_wrappers import GPyModelWrapper

In [ ]:
model_dynamics_emulated_emukit = GPyModelWrapper(model_dynamics_emulated_gpy, n_restarts=5)
model_dynamics_emulated_emukit.optimize()

We set the acquisition function to be expected improvement.

In [ ]:
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement

In [ ]:
acquisition = ExpectedImprovement(model_emukit)

And we set up the main loop for the Bayesian optimization.

In [ ]:
from emukit.bayesian_optimization.loops.bayesian_optimization_loop import BayesianOptimizationLoop

In [ ]:
bo = BayesianOptimizationLoop(space, model_dynamics_emulated_emukit, acquisition=acquisition)
bo.run_loop(target_function_emulator, 50)
results = bo.get_results()

In [ ]:
_, _, _, frames = mc.run_simulation(env, np.atleast_2d(results.minimum_location), render=True)
anim=mc.animate_frames(frames, 'Best controller using the emulator of the dynamics')

In [ ]:
from IPython.core.display import HTML

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
mc.save_frames(frames, 
                  diagrams='./uq', 
                  filename='mountain-car-emulated.html')

<iframe src="https://mlatcl.github.io/mlphysical/./slides/diagrams//uq/mountain-car-emulated.html" width="600" height="450" allowtransparency="true" frameborder="0">
</iframe>

Figure: <i>Mountain car controller learnt through emulation. Here 500
calls to the simulator are used to fit the controller rather than 37,500
calls to the simulator required in the standard learning.</i>

And the problem is again solved, but in this case, we have replaced the
simulator of the car dynamics by a Gaussian process emulator that we
learned by calling the dynamics simulator only 500 times. Compared to
the 37,500 calls that we needed when applying Bayesian optimization
directly on the simulator this is a significant improvement. Of course,
in practice the car dynamics are very simple for this example.

## Mountain Car: Multi-Fidelity Emulation

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_uq/includes/mountain-car-multi-fidelity-introduction.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_uq/includes/mountain-car-multi-fidelity-introduction.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

In some scenarios we have simulators of the same environment that have
different fidelities, that is that reflect with different level of
accuracy the dynamics of the real world. Running simulations of the
different fidelities also have a different cost: high-fidelity
simulations are typically more expensive the low-fidelity. If we have
access to these simulators, we can combine high and low-fidelity
simulations under the same model.

So, let’s assume that we have two simulators of the mountain car
dynamics, one of high fidelity (the one we have used) and another one of
low fidelity. The traditional approach to this form of multi-fidelity
emulation is to assume that $$
f_i\left(\mathbf{ x}\right) = \rho f_{i-1}\left(\mathbf{ x}\right) + \delta_i\left(\mathbf{ x}\right),
$$ where $f_{i-1}\left(\mathbf{ x}\right)$ is a low-fidelity simulation
of the problem of interest and $f_i\left(\mathbf{ x}\right)$ is a higher
fidelity simulation. The function $\delta_i\left(\mathbf{ x}\right)$
represents the difference between the lower and higher fidelity
simulation, which is considered additive. The additive form of this
covariance means that if $f_{0}\left(\mathbf{ x}\right)$ and
$\left\{\delta_i\left(\mathbf{ x}\right)\right\}_{i=1}^m$ are all
Gaussian processes, then the process over all fidelities of simulation
will be a joint Gaussian process.

But with deep Gaussian processes we can consider the form $$
f_i\left(\mathbf{ x}\right) = g_{i}\left(f_{i-1}\left(\mathbf{ x}\right)\right) + \delta_i\left(\mathbf{ x}\right),
$$ where the low fidelity representation is nonlinearly transformed by
$g(\cdot)$ before use in the process. This is the approach taken in P.
Perdikaris et al. (2017). But once we accept that these models can be
composed, a highly flexible framework can emerge. A key point is that
the data enters the model at different levels and represents different
aspects. For example, these correspond to the two fidelities of the
mountain car simulator.

We start by sampling both at 250 random input locations.

In [ ]:
import gym

In [ ]:
env = gym.make('MountainCarContinuous-v0')

In [ ]:
from emukit.core import ContinuousParameter, ParameterSpace

In [ ]:
position_dynamics_domain = [-1.2, +0.6]
velocity_dynamics_domain = [-0.07, +0.07]
action_dynamics_domain = [-1, +1]

space_dynamics = ParameterSpace(
          [ContinuousParameter('position_dynamics_parameter', *position_dynamics_domain), 
           ContinuousParameter('velocity_dynamics_parameter', *velocity_dynamics_domain),
           ContinuousParameter('action_dynamics_parameter', *action_dynamics_domain)])

Next, we evaluate the high and low fidelity simualtors at those
locations.

In [ ]:
import numpy as np
import mountain_car as mc

In [ ]:
n_points = 250
d_position_hf = np.zeros((n_points, 1))
d_velocity_hf = np.zeros((n_points, 1))
d_position_lf = np.zeros((n_points, 1))
d_velocity_lf = np.zeros((n_points, 1))

# --- Collect high fidelity points
for i in range(0, n_points):
    d_position_hf[i], d_velocity_hf[i] = mc.simulation(x_random[i, :])

# --- Collect low fidelity points  
for i in range(0, n_points):
    d_position_lf[i], d_velocity_lf[i] = mc.low_cost_simulation(x_random[i, :])

## Prime Air

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_ai/includes/prime-air-system.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_ai/includes/prime-air-system.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

One project where the components of machine learning and the physical
world come together is Amazon’s Prime Air drone delivery system.

Automating the process of moving physical goods through autonomous
vehicles completes the loop between the ‘bits’ and the ‘atoms.’ In other
words, the information and the ‘stuff.’ The idea of the drone is to
complete a component of package delivery, the notion of last mile
movement of goods, but in a fully autonomous way.

<svg viewBox="0 0 200 200" style="width:15%">

<defs> <clipPath id="clip1">

<style>
circle {
  fill: black;
}
</style>

<circle cx="100" cy="100" r="100"/> </clipPath> </defs>

<title>

Gur Kimchi

</title>

<image preserveAspectRatio="xMinYMin slice" width="100%" xlink:href="https://mlatcl.github.io/mlphysical/./slides/diagrams//people/gur-kimchi.png" clip-path="url(#clip1)"/>

</svg>
<svg viewBox="0 0 200 200" style="width:15%">

<defs> <clipPath id="clip2">

<style>
circle {
  fill: black;
}
</style>

<circle cx="100" cy="100" r="100"/> </clipPath> </defs>

<title>

Paul Viola

</title>

<image preserveAspectRatio="xMinYMin slice" width="100%" xlink:href="https://mlatcl.github.io/mlphysical/./slides/diagrams//people/paul-viola.png" clip-path="url(#clip2)"/>

</svg>
<svg viewBox="0 0 200 200" style="width:15%">

<defs> <clipPath id="clip3">

<style>
circle {
  fill: black;
}
</style>

<circle cx="100" cy="100" r="100"/> </clipPath> </defs>

<title>

David Moro

</title>

<image preserveAspectRatio="xMinYMin slice" width="100%" xlink:href="https://mlatcl.github.io/mlphysical/./slides/diagrams//people/david-moro.png" clip-path="url(#clip3)"/>

</svg>

In [ ]:
from IPython.lib.display import YouTubeVideo
YouTubeVideo('3HJtmx5f1Fc')

Figure: <i>An actual Santa’s sleigh. Amazon’s new delivery drone.
Machine learning algorithms are used across various systems including
sensing (computer vision for detection of wires, people, dogs etc) and
piloting. The technology is necessarily a combination of old and new
ideas. The transition from vertical to horizontal flight is vital for
efficiency and requires sophisticated machine learning to achieve.</i>

As Jeff Wilke (who was CEO of Amazon Retail at the time) [announced in
June
2019](https://blog.aboutamazon.com/transportation/a-drone-program-taking-flight)
the technology is ready, but still needs operationalization including
e.g. regulatory approval.

In [ ]:
from IPython.lib.display import YouTubeVideo
YouTubeVideo('wa8DU-Sui8Q')

Figure: <i>Jeff Wilke (CEO Amazon Consumer) announcing the new drone at
the Amazon 2019 re:MARS event alongside the scale of the Amazon supply
chain.</i>

> When we announced earlier this year that we were evolving our Prime
> two-day shipping offer in the U.S. to a one-day program, the response
> was terrific. But we know customers are always looking for something
> better, more convenient, and there may be times when one-day delivery
> may not be the right choice. Can we deliver packages to customers even
> faster? We think the answer is yes, and one way we’re pursuing that
> goal is by pioneering autonomous drone technology.

> Today at Amazon’s re:MARS Conference (Machine Learning, Automation,
> Robotics and Space) in Las Vegas, we unveiled our latest Prime Air
> drone design. We’ve been hard at work building fully electric drones
> that can fly up to 15 miles and deliver packages under five pounds to
> customers in less than 30 minutes. And, with the help of our
> world-class fulfillment and delivery network, we expect to scale Prime
> Air both quickly and efficiently, delivering packages via drone to
> customers within months.

The 15 miles in less than 30 minutes implies air speed velocities of
around 50 kilometers per hour.

> Our newest drone design includes advances in efficiency, stability
> and, most importantly, in safety. It is also unique, and it advances
> the state of the art. How so? First, it’s a hybrid design. It can do
> vertical takeoffs and landings – like a helicopter. And it’s efficient
> and aerodynamic—like an airplane. It also easily transitions between
> these two modes—from vertical-mode to airplane mode, and back to
> vertical mode.

> It’s fully shrouded for safety. The shrouds are also the wings, which
> makes it efficient in flight.

<img class="" src="https://mlatcl.github.io/mlphysical/./slides/diagrams//ai/amazon-prime-air-remars-june-2019.jpg" style="width:80%">

Figure: <i>Picture of the drone from Amazon Re-MARS event in 2019.</i>

> Our drones need to be able to identify static and moving objects
> coming from any direction. We employ diverse sensors and advanced
> algorithms, such as multi-view stereo vision, to detect static objects
> like a chimney. To detect moving objects, like a paraglider or
> helicopter, we use proprietary computer-vision and machine learning
> algorithms.

> A customer’s yard may have clotheslines, telephone wires, or
> electrical wires. Wire detection is one of the hardest challenges for
> low-altitude flights. Through the use of computer-vision techniques
> we’ve invented, our drones can recognize and avoid wires as they
> descend into, and ascend out of, a customer’s yard.

## Thanks!

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/multifidelity.gpp.markdown" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/multifidelity.gpp.markdown', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

For more information on these subjects and more you might want to check
the following resources.

-   twitter: [@lawrennd](https://twitter.com/lawrennd)
-   podcast: [The Talking Machines](http://thetalkingmachines.com)
-   newspaper: [Guardian Profile
    Page](http://www.theguardian.com/profile/neil-lawrence)
-   blog:
    [http://inverseprobability.com](http://inverseprobability.com/blog.html)

## References

Álvarez, M.A., 2011. Convolved Gaussian process priors for multivariate
regression with applications to dynamical systems (PhD thesis). School
of Computer Science, University of Manchester, Manchester, UK.

Álvarez, M.A., Luengo, D., Lawrence, N.D., 2013. Linear latent force
models using Gaussian processes. IEEE Transactions on Pattern Analysis
and Machine Intelligence 35, 2693–2705.
<https://doi.org/10.1109/TPAMI.2013.86>

Álvarez, M.A., Rosasco, L., Lawrence, N.D., 2012. Kernels for
vector-valued functions: A review. Foundations and Trends in Machine
Learning 4, 195–266. <https://doi.org/10.1561/2200000036>

Damianou, A., 2015. Deep Gaussian processes and variational propagation
of uncertainty (PhD thesis). University of Sheffield.

Damianou, A., Lawrence, N.D., 2013. Deep Gaussian processes. pp.
207–215.

Kennedy, M.C., O’Hagan, A., 2000. Predicting the output from a complex
computer code when fast approximations are available. Biometrika 87,
1–13.

Lawrence, N.D., Moore, A.J., 2007. Hierarchical Gaussian process latent
variable models. pp. 481–488.

Lawrence, N.D., Sanguinetti, G., Rattray, M., n.d. Modelling
transcriptional regulation using Gaussian processes. pp. 785–792.

Perdikaris, P., Raissi, M., Damianou, A., Lawrence, N.D., Karniadakis,
G.E., 2017. Nonlinear information fusion algorithms for data-efficient
multi-fidelity modelling. Proceedings of the Royal Society of London A:
Mathematical, Physical and Engineering Sciences 473.
<https://doi.org/10.1098/rspa.2016.0751>

Perdikaris, Paris, Raissi, M., Damianou, A., Lawrence, N.D., Karnidakis,
G.E., 2017. Nonlinear information fusion algorithms for data-efficient
multi-fidelity modelling. Proc. R. Soc. A 473.
<https://doi.org/10.1098/rspa.2016.0751>

Shahriari, B., Swersky, K., Wang, Z., Adams, R.P., de Freitas, N., 2016.
Taking the human out of the loop: A review of Bayesian optimization.
Proceedings of the IEEE 104, 148–175.
<https://doi.org/10.1109/JPROC.2015.2494218>